#### Example 4-1. Phone book lookup with a list

In [1]:
def find_phonenumber(phonebook, name):
    for n, p in phonebook:
        if n == name:
            return p
    return none

In [2]:
phonebook = [
    ("John Doe", "555-555-5555"),
    ("Albert Einstein", "212-555-5555")
]

print("John Doe's phone number is", find_phonenumber(phonebook, "John Doe"))

John Doe's phone number is 555-555-5555


In [3]:
%timeit find_phonenumber(phonebook, "John Doe")

160 ns ± 7.44 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


#### Example 4-2. Phone book lookup with a dictionary

In [4]:
phonebook = {
    "John Doe": "555-555-5555",
    "Albert Einstein": "212-555-5555"
}

print("John Doe's phone number is", phonebook["John Doe"])

John Doe's phone number is 555-555-5555


In [5]:
%timeit phonebook["John Doe"]

45.2 ns ± 1.85 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


#### Result
Lookup with a dictionary is 4x faster than that with a list.

<hr>

#### Example 4-3. Finding unique names with lists and sets

In [6]:
def list_unique_names(phonebook):
    unique_names = []
    
    for name, phonenumber in phonebook:
        is_added = True
        first_name, last_name = name.split(" ", 1)
        for unique in unique_names:
            if unique == first_name:
                is_added = False
                break
        if is_added:
            unique_names.append(first_name)
    
    return len(unique_names)

In [7]:
def set_unique_names(phonebook):
    unique_names = set()
    
    for name, phonenumber in phonebook:
        first_name, last_name = name.split(" ", 1)
        unique_names.add(first_name)
    
    return len(unique_names)

In [8]:
phonebook = [
    ("John Doe", "555-555-5555"),
    ("Albert Einstein", "212-555-5555"),
    ("John Murphey", "202-555-5555"),
    ("Albert Rutherford", "647-555-5555"),
    ("Elaine Bodian", "301-555-5555")
]

In [9]:
print("Number of unique names from set method:", set_unique_names(phonebook))
print("Number of unique names from list method:", list_unique_names(phonebook))

Number of unique names from set method: 3
Number of unique names from list method: 3


In [10]:
%timeit set_unique_names(phonebook)

1.72 µs ± 52.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [11]:
%timeit list_unique_names(phonebook)

1.81 µs ± 66.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


#### Result
When the size of data is small, process time is almost similar.

<hr>

#### Problem
Is there a difference in processing time as the size of the list grows?

In [12]:
import pandas as pd

In [13]:
df = pd.read_csv("../data/phone_number.csv")

In [14]:
df.head()

,name,phone number
0,John Cline,251-690-503
1,Malaki Glenn,322-337-492
2,Julie Pitts,756-914-220
3,Samanta Marks,363-663-339
4,Maisie Salgado,861-184-838


In [15]:
records = df.to_records(index=False)
phonebook = list(records)
phonebook[0:5]

[('John Cline', '251-690-503'),
 ('Malaki Glenn', '322-337-492'),
 ('Julie Pitts', '756-914-220'),
 ('Samanta Marks', '363-663-339'),
 ('Maisie Salgado', '861-184-838')]

In [16]:
print("Number of unique names from set method:", set_unique_names(phonebook))
print("Number of unique names from list method:", list_unique_names(phonebook))

Number of unique names from set method: 903
Number of unique names from list method: 903


In [17]:
%timeit set_unique_names(phonebook)

3.12 ms ± 284 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
%timeit list_unique_names(phonebook)

15.3 ms ± 917 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Result
The set algorithm gave us a 3x speedup! This is because the loop counts are different. 
- set_unique_names -> O(n)
- list_unique_names -> O(n log n)

## How Do Dictionaries and Sets Work?

#### Example 4-4. Dictionary lookup sequence

In [19]:
def index_sequence(key, mask=0b111, PERTURB_SHIFT=5):
    perturb = hash(key)
    i = perturb & mask
    yield i
    while True:
        perturb >>= PERTURB_SHIFT
        i = ((i << 2) + i + perturb + 1)
        yield i & mask

In [20]:
from itertools import islice

class TestHash(object):
    def __init__(self, test_hash):
        self.test_hash = test_hash

    def __hash__(self):
        return self.test_hash

    
probe_index_iter = index_sequence(TestHash(0b111))
indexes = islice(probe_index_iter, 12)
list(indexes)

[7, 4, 5, 2, 3, 0, 1, 6, 7, 4, 5, 2]

#### Result
When insertion of a key causes a collision, a new index is calculated using the scheme in Example 4-4. (Iterator)

### Hash Functions and Entropy

In [21]:
class Point(object):
    def __init__(self, x, y):
        self.x, self.y = x, y

In [22]:
p1 = Point(1,1)
p2 = Point(1,1)

print(set([p1, p2]))
Point(1,1) in set([p1, p2])

{<__main__.Point object at 0x129e13dd8>, <__main__.Point object at 0x129e130f0>}


False

#### Result
Two instances (p1 and p2) of a class are generally different and should not collide in a hash table. So both instances would result in individual entries.

<hr>

#### Problem
What if  we don't want to add the instance in a set, when an instance with the same property values already exists in a set?

In [23]:
class Point(object):
    def __init__(self, x, y):
        self.x, self.y = x, y
    
    def __hash__(self):
        return hash((self.x, self.y))
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

In [24]:
p1 = Point(1,1)
p2 = Point(1,1)

print(set([p1, p2]))
Point(1,1) in set([p1, p2])

{<__main__.Point object at 0x129e16ef0>}


True

#### Example 4-7. Timing differences between good and bad hashing functions

In [25]:
import string
import timeit

class BadHash(str):
    def __hash__(self):
        return 42
    
class GoodHash(str):
    def __hash__(self):
        """
        This is a slightly optimized version of twoletter_hash
        """
        return ord(self[1]) + 26 * ord(self[0])

In [26]:
baddict = set()
gooddict = set()

for i in string.ascii_lowercase:
    for j in string.ascii_lowercase:
        key = i + j
        baddict.add(BadHash(key))
        gooddict.add(BadHash(key))

In [27]:
badtime = timeit.repeat(
    "key in baddict",
    setup = "from __main__ import baddict, BadHash; key = BadHash('zz')",
    repeat = 3,
    number = 1000000,
)

goodtime = timeit.repeat(
    "key in gooddict",
    setup = "from __main__ import gooddict, GoodHash; key = GoodHash('zz')",
    repeat = 3,
    number = 1000000,
)

In [28]:
print("Min lookup time for baddict: ", min(badtime))
print("Min lookup time for gooddict: ", min(goodtime))

Min lookup time for baddict:  14.968249684
Min lookup time for gooddict:  0.27232210199998974


#### Result
We can reduce the lookup time by setting up an efficient hash function.

## Dictionaries and Namespaces

#### Example 4-8. Namespace lookups

In [29]:
import math
from math import sin

def test1(x):
    return math.sin(x)

def test2(x):
    return sin(x)

def test3(x, local_sin=math.sin):
    return local_sin(x)

def test4(x, local_sin=sin):
    return local_sin(x)

In [30]:
%timeit test1(123456)

171 ns ± 7.26 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [31]:
%timeit test2(123456)

137 ns ± 6.42 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [32]:
%timeit test3(123456)

149 ns ± 4.78 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [33]:
%timeit test4(123456)

143 ns ± 4.41 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


#### Result
- process speed: locals() > globals() > __builtin__
- In test3 function, which assigns a global variable as an argument, so it must be taking a lot of time to process.

#### Example 4-10. Effects of slow namespace lookups in loops

In [34]:
from math import sin

def tight_loop_slow(iterations):
    result = 0
    for i in range(iterations):
        # this call to sin requires a global lookup
        result += sin(i)
        
def tight_loop_fast(iterations):
    result = 0
    local_sin=sin
    for i in range(iterations):
        # this call to local_sin requires a local lookup
        result += local_sin(i)

In [35]:
%timeit tight_loop_slow(10000000)

1.27 s ± 37.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
%timeit tight_loop_fast(10000000)

1.05 s ± 20.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
